# 5 задача
## Решить задачу Дирихле для уравнения Пуассона методом установления с упорядоченным набором оптимальных чебышевских параметров

### Постановка задачи (вариант №15)

#### Уравнение Пуассона:

\begin{equation}
\left.
  \begin{array}{ccc}
      {\partial^2u \over \partial x^2} + {\partial^2u \over \partial y^2} = -25 \pi^2 \:\mathrm{cos}(3\pi x) \:\mathrm{sin}(4\pi y)\\
      u \mid_Г = 0
  \end{array}
\right\}
\end{equation}

\begin{equation}
-{1 \over 2} \leq x \leq {1 \over 2}; \; \; \; 0 \leq y \leq 1.
\end{equation}

#### Аналитическое решение очевидно:
\begin{equation}
u(x, y) = \mathrm{cos}(3\pi x)\: \mathrm{sin}(4\pi y)
\end{equation}

Точность численного решения можно оценивать по 1 метрике:
\begin{equation}
\bigg\vert \Delta u \bigg\vert_1 = \underset{k, l}{\mathrm{max}} \{\:\mid u_{k,l}^{model} - u_{k,l}^{theory}\mid\: \}
\end{equation}

In [10]:
from math import *
import numpy as np
import matplotlib as mpl

def f(x, y):
    return -25*math.pi**2*math.cos(3*math.pi*x)*math.sin(4*math.pi*y)

def F(Nx,Ny):
    return np.array([[f( x/(Nx-1)-0.5, y/(Ny-1) ) for x in range(0,Nx)] for y in range(0,Ny)])
    

In [4]:
def del2(M,Nx,Ny):
    dx = 1./Nx
    dy = 1./Ny
    rows, cols = M.shape
    #dx = dx * np.ones ((1, cols - 1))
    #dy = dy * np.ones ((rows-1, 1))

    mr, mc = M.shape
    D = np.zeros ((mr, mc))

    if (mr >= 4):
        ## x direction
        ## left and right boundary
        D[:,0] = (2*M[:,0] - 5*M[:,1] + 4*M[:,2] - M[:,3]) / dx**2
        D[:,mr-1] = (-M[:,mr-4] + 4*M[:,mr-3] - 5*M[:,mr-2] + 2*M[:,mr-1]) / dx**2

        ## interior points
        tmp1 = D[:,1:mr-1] 
        tmp2 = (M[:,2:mr] - 2*M[:,1:mr-1] + M[:,0:mr-2])
        D[:,1:mr-1] = tmp1 + tmp2 / dx**2

    if (mc >= 4):
        ## y direction
        ## top and bottom boundary
        D[0,:] = D[0,:]  + (2*M[0,:] - 5*M[1,:] + 4*M[2,:] - M[3,:]) / dy**2
        D[mc-1,:] = D[mc-1,:] + (-M[mc-4,:] + 4*M[mc-3,:] - 5*M[mc-2,:] + 2*M[mc-1,:]) / dy**2

        ## interior points
        tmp1 = D[1:mc-1,:] 
        tmp2 = (M[2:mc,:] - 2*M[1:mc-1,:] + M[0:mc-2,:])
        D[1:mc-1, :] = tmp1 + tmp2 / dy**2

    return D


In [6]:
N = 19 # число шагов по времени
chebyshev = np.array([1,37,15,23,7,31,9,29,3,35,13,25,5,33,11,27,17,21,19]) # набор параметров для N = 19
chebyshev.size

19

In [11]:
# шаг по времени буду выбирать упорядоченным оптимальным набором чебышевских параметров через функцию:
def choose_t(n):
    maxmu = 4*(cos(pi/2/Nx)**2*Nx**2 + cos(pi/2/Ny)**2*Ny**2)
    minmu = 4*(sin(pi/2/Nx)**2*Nx**2 + sin(pi/2/Ny)**2*Ny**2)
    denominator = maxmu + minmu + (maxmu - minmu)*cos(pi*(2*n-1)/2/N)
    return 2 / denominator

In [8]:
# нахождение приближения на следующем шаге
def step(M,Nx,Ny,dt,Fpart):
    new = M + (del2(M,Nx,Ny) + Fpart) * dt
    return new

In [15]:
Nx, Ny = 81, 81
sol = np.zeros((Nx,Ny))
Fpart = F(Nx,Ny)

for i in range(0,N):
    sol = step(sol,Nx,Ny,choose_t(chebyshev[i]),Fpart)
sol6561 = sol

In [18]:
exactsol = np.array([[math.cos(3*math.pi*i/10) * math.sin(4*math.pi*j/10) for j in range(0,11)] for i in range(-5,6)])

In [27]:
exactsol

array([[-0.00000000e+00, -1.74706248e-16, -1.07974399e-16,
         1.07974399e-16,  1.74706248e-16,  4.49927935e-32,
        -1.74706248e-16, -1.07974399e-16,  1.07974399e-16,
         1.74706248e-16,  8.99855870e-32],
       [-0.00000000e+00, -7.69420884e-01, -4.75528258e-01,
         4.75528258e-01,  7.69420884e-01,  1.98152015e-16,
        -7.69420884e-01, -4.75528258e-01,  4.75528258e-01,
         7.69420884e-01,  3.96304029e-16],
       [-0.00000000e+00, -9.04508497e-01, -5.59016994e-01,
         5.59016994e-01,  9.04508497e-01,  2.32941664e-16,
        -9.04508497e-01, -5.59016994e-01,  5.59016994e-01,
         9.04508497e-01,  4.65883327e-16],
       [-0.00000000e+00, -2.93892626e-01, -1.81635632e-01,
         1.81635632e-01,  2.93892626e-01,  7.56873346e-17,
        -2.93892626e-01, -1.81635632e-01,  1.81635632e-01,
         2.93892626e-01,  1.51374669e-16],
       [ 0.00000000e+00,  5.59016994e-01,  3.45491503e-01,
        -3.45491503e-01, -5.59016994e-01, -1.43965866e-16,
  

In [41]:
sol6561

array([[-3.41015297e-05,  4.32825432e-05, -1.20563867e-05,
        -3.27635268e-05, -5.29056801e-05, -4.82771633e-05,
        -1.36932051e-05, -3.30944373e-05, -2.44136911e-05,
         4.02658239e-05,  1.81433400e-05],
       [-2.96815604e-05, -7.69411638e-01, -4.75495013e-01,
         4.75497245e-01,  7.69450384e-01,  1.16406908e-05,
        -7.69432053e-01, -4.75496554e-01,  4.75512915e-01,
         7.69432330e-01,  3.70878566e-05],
       [ 1.83240000e-05, -9.04510975e-01, -5.59062763e-01,
         5.58965038e-01,  9.04531648e-01,  3.28332100e-05,
        -9.04562611e-01, -5.59005681e-01,  5.59049966e-01,
         9.04508136e-01,  2.64248108e-05],
       [-4.36784785e-05, -2.93936602e-01, -1.81591837e-01,
         1.81662002e-01,  2.93907896e-01,  2.50198583e-05,
        -2.93925515e-01, -1.81622822e-01,  1.81650558e-01,
         2.93839746e-01,  2.86134170e-06],
       [-2.58362476e-05,  5.58994824e-01,  3.45460466e-01,
        -3.45514298e-01, -5.58985923e-01, -1.05544156e-05,
  

In [20]:
# макс и мин значения на сетке
print(max((exactsol).flatten()))
print(min((exactsol).flatten()))

0.9739489492010454
-0.9739489492010454


In [40]:
# наибольшая относительная ошибка
max((abs(exactsol-sol6561)).flatten())

5.47792482165077e-05